# Resumo do código

### <u>Código que gera os ficheiros para estudo do comportamento da Delta</u>
---
O objectivo é receber dados da Delta e devolver um conjunto de métricas para prever roturas. Devolve um ficheiro que pode entrar no código 1 para juntar aos dados dos ninjas.

---
- Inputs

> __Dados completos da Delta em pastas de ficheiros__ (de azul a verde)
> - Stocks e trânsito, Sellout do dia anterior

                    ou

> __Ficheiro já completo__ (de vermelho a verde)
> - Stocks e trânsito, Sellout do dia anterior

- Outputs

> __Ficheiro com produtos em causa__ em formato Long

> __Métricas novas:__
> - Roturas de Stock e Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - MSA (média de sellouts 10 dias antes)
> - STK (Stock disponível + trânsito)
> - (Novo) Balanço médio, mediano, liberal e conservador 
> - (Novo) Dias para a rotura de stock e de prateleira


In [1]:
%%time
import pandas as pd
import os
import glob
import numpy as np
import datetime
from IPython.display import Audio

def escrever_csv(dfa, nome):
    dfa.to_csv(nome+'.csv', index=False)

CPU times: total: 344 ms
Wall time: 350 ms


---

---

#  <span style="color:red"><u>Ler Ficheiro Completo</u> </span>

In [2]:
%%time
#Ler o ficheiro

df_2022 = pd.read_csv('D:\\B&N Dados\\Delta\\Stocks\\Stocks2022\\Stocks_Delta_2022_Limpo.csv')

CPU times: total: 16.8 s
Wall time: 19.5 s


---

- Produtos específicos

In [48]:
%%time
# Ler ficheiro dos produtos para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\produtos.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()

# Alterar o dataframe para apenas incluir os produtos em causa
dfInicial = df_2022[df_2022["DESC_ARTIGO"].isin(produtos)].copy()

CPU times: total: 672 ms
Wall time: 1.94 s


In [77]:
resto = dfInicial.columns.difference(["DESC_ARTIGO", "EAN",'SELLOUT_1_Dias_Antes' , 'STOCK_1_Dias_Antes','STORE_NAME'])

In [70]:
dfNovo = pd.DataFrame(dfInicial["DATA"].unique().tolist(), columns=["DATA"])

dfModelo=dfNovo.copy()

In [72]:
lojas = dfInicial["STORE"].unique().tolist()

# Criar uma lista de dfs, vai ter o número de alturas do dia certo
dfs = [dfModelo.assign(STORE=l) for l in lojas]

# Juntar tudo no mesmo dataframe
df_Mergir = pd.concat(dfs, ignore_index=True)

In [78]:
# Dataframe mergido no fim
dfMeio=df_Mergir.copy()
resto = dfInicial.columns.difference(["DESC_ARTIGO", "EAN",'SELLOUT_1_Dias_Antes' , 'STOCK_1_Dias_Antes','STORE_NAME'])

for coluna in produtos:       
    
    #Cada dfStocks é um dataframe com o produto específico
    dfStocks=dfInicial[dfInicial["DESC_ARTIGO"]==coluna][resto]   #Dados Ninjas, seleccionar colunas do café específico
 
    #Juntar dados de acordo com o dia e a loja
    dfMeio=pd.merge(dfMeio, dfStocks, how="left", on = ["DATA", "STORE"])  
    
    #Mudar nomes para ser adaptado a cada produto
    dfMeio=dfMeio.rename(columns={"STOCK": "STOCK %s" % coluna, "PRES_STOCK":"PRESLINEAR %s" % coluna, "INTRANSIT":"INTRANSIT %s" % coluna, "EXPECTED":"EXPECTED %s" % coluna, "SELLOUT":"SELLOUT %s" % coluna})                       #Nomear coluna nova
    
    

In [80]:
dfFinal = dfMeio.copy()

# Colunas de métricas interessantes

> - ROTURA

In [82]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

for i in produtos:
    rotura  = "ROTURA %s" % i
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    
    dfFinal[rotura] = np.where((dfFinal[stock] <= 0) & (dfFinal[preslinear] > 0), 1, 0)


> - PRÉ_ROTURA

In [83]:
# Definir coluna de rotura (se stock menor ou igual a 0)

for i in produtos:
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    pre_rotura= "PRÉ-ROTURA %s" % i
    
    dfFinal[pre_rotura] = (dfFinal[stock] < dfFinal[preslinear]).astype(int)

# Métricas até 10 dias antes:

- INSTRANSIT
- EXPECTED
- SELLOUT
- CICLOS
- Dias para Rotura
- Adequação

In [29]:
# Quantos dias antes:

diaI=1         #dia inicial
diaF=10        #dia final

> Função

In [30]:
# Função para colunas de dias anteriores
def dias(df, dia, coluna):         #dia é quantos dias antes
    
    #
    a=int(dia)

    valores = df.groupby(['DESC_ARTIGO', 'STORE'])[coluna].transform(lambda x: x.shift(a))
    valores[:a] = np.nan
    
    df.loc[:,'%s_%s_Dias_Antes' % (coluna, a)] = valores

> - SELLOUTS

In [31]:
%%time
# Usar função para sellouts até 10 dias antes

for i in range(diaI+1, diaF+1):
    dias(dfFinal, i, "SELLOUT")

CPU times: total: 6.39 s
Wall time: 6.84 s


> - STOCKS

In [32]:
# Usar função para Stocks até 10 dias antes

for i in range(diaI+1, diaF+1):
    dias(dfFinal, i, "STOCK")

> - INTRANSIT e EXPECTED

In [33]:
# Usar função para Trânsito até 10 dias antes

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "INTRANSIT")
    
for i in range(diaI, diaF+1):
    dias(dfFinal, i, "EXPECTED")

> - STK

In [34]:
# STK do dia = soma dos stocks em loja com os stocks em trânsito no próprio dia

dfFinal["STK"] = dfFinal["STOCK"] + dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "STK")

> - MSA

In [35]:
# MSA do dia = média dos sellouts dos 10 dias anteriores ao dia em causa

dfFinal["MSA10"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).mean())
dfFinal["MSA10Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).std())

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "MSA10")

    
dfFinal["MSA20"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20).mean())
dfFinal["MSA20Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20).std())
  
for i in range(diaI, diaF+1):
    dias(dfFinal, i, "MSA20")

> - CICLOS

In [36]:
# Coluna de Ciclos de reposição

for i in produtos:
    ciclos = "CICLO %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    dfFinal[ciclos]=dfFinal[stock]/dfFinal[linear]

> - Dias para rotura de Stock

In [37]:
dfFinal = dfFinal.copy()
# Dias para a rotura mas com o Sellout médio (móvel) dos últimos 10 dias 
dfFinal["Dias_para_Rotura_Stock"] = dfFinal["STOCK"] / dfFinal["MSA10"]

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "Dias_para_Rotura_Stock")
    
for i in produtos:
    stock  = "STOCK %s" % i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" % i
    dias_rotura = "Dias_para_Rotura %s" % i
    
    dfFinal[dias_rotura] = dfFinal[stock]/dfFinal[sellout]

> - Dias para rotura de Linear

In [38]:
# Definir a métrica: Preslinear / med(Sellouts 10 dias)
dfFinal['Dias_Duração_Linear'] = dfFinal["PRES_STOCK"] / dfFinal["MSA10"]

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "Dias_Duração_Linear")
    
for i in produtos:
    linear = "PRESLINEAR %s" % i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" % i
    dias_rotura_prat = "Dias_para_Rotura_Linear %s" % i
    
    dfFinal[dias_rotura_prat] = dfFinal[linear] / dfFinal[sellout]

> - Adequação de Stock

In [39]:
# Coluna de adequação de stock


dfFinal["Adequação"]= np.where(dfFinal["CICLOS"] > 1.1, "Stock Suficiente", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]>=dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Adequado", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]<dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Desadequado", 
                      "")))

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "Adequação")
    

# Define the mappings of numbers to strings
mapping = {1: "Stock Suficiente",
           2: "Stock Insuf c Forn Adequado",
           3: "Stock Insuf c Forn Desadequado"}

for i in produtos:
    adequa = "ADEQUAÇÃO %s" % i
    ciclos = "CICLO %s" % i
    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    
    dfFinal[adequa] = np.where(dfFinal[ciclos] > 1.1, 1,
                      np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito] + dfFinal[esperado] + dfFinal[stock] >= dfFinal[linear]), 2,
                      np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito] + dfFinal[esperado] + dfFinal[stock] < dfFinal[linear]), 3, 
                      np.nan)))
    
    # Map the numbers to the corresponding strings
    dfFinal[adequa] = dfFinal[adequa].map(mapping)

>- Balance: sellout / soma stock disponível mais transito.

In [40]:
# Colunas de balanço


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance"] =  dfFinal["MSA10"] / dfFinal["STK"]

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal["Balance"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance", i)] = valores
    
    


>- Balance optimizado

In [41]:


    
dfFinal["Balance_Optimized"] = np.where((dfFinal["MSA10Dp"] / dfFinal["MSA10"]) * 100 > 100, dfFinal["MSA20"] / dfFinal["STK"],
                                dfFinal["MSA10"] / dfFinal["STK"])   

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal["Balance_Optimized"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Optimized", i)] = valores

> - Mediano é com o mínimo dos ultimos 10 dias

In [42]:
# Coluna de adequação de stock


# MSA_med do dia = mediana dos sellouts dos 10 dias anteriores (exclui o próprio dia)
# dfFinal["MdSA"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).median())


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance_Mediano"] =  dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).median()) / dfFinal["STK"]

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal["Balance_Mediano"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Mediano", i)] = valores



> - Liberal é com o mínimo dos ultimos 10 dias

In [43]:
# Liberal


# MSA do dia = média dos stocks dos 10 dias anteriores (exclui o próprio dia)
#dfFinal["LSA"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).min())


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance_Liberal"] =  dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).min()) / dfFinal["STK"]

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal["Balance_Liberal"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Liberal", i)] = valores



> - Conservador é com o máximo dos ultimos 10 dias

In [44]:
# Conservador

# MSA_max do dia = máximo dos stocks dos 10 dias anteriores (exclui o próprio dia)
#dfFinal["CSA"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).max())
 
    
# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance_Conservador"] =  dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).max()) / dfFinal["STK"]

for i in range(diaI, diaF+1):
    
    # Multiplicar os balances pelo número de dias+1 antes do dia actual
    valores = (i+1) * dfFinal["Balance_Conservador"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Conservador", i)] = valores



In [28]:
dfFinal[["DATA","STORE","DESC_ARTIGO","STOCK","INTRANSIT","EXPECTED","SELLOUT","Balance","Balance_4_Dias_Antes","Balance_Liberal","Balance_Liberal_4_Dias_Antes","Balance_Mediano_4_Dias_Antes","Balance_Conservador_4_Dias_Antes"]].tail(20)

,DATA,STORE,DESC_ARTIGO,STOCK,INTRANSIT,EXPECTED,SELLOUT,Balance,Balance_4_Dias_Antes,Balance_Liberal,Balance_Liberal_4_Dias_Antes,Balance_Mediano_4_Dias_Antes,Balance_Conservador_4_Dias_Antes
11988342,2023-04-21,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,38.0,0,0,5.0,0.128947,0.568627,0.026316,0.098039,0.490196,1.078431
11988343,2023-04-22,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,33.0,0,0,9.0,0.151515,0.540816,0.030303,0.102041,0.459184,1.122449
11988344,2023-04-23,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,24.0,0,0,5.0,0.233333,0.600000,0.041667,0.111111,0.500000,1.222222
11988345,2023-04-24,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,19.0,0,6,2.0,0.224000,0.756410,0.040000,0.256410,0.641026,1.410256
11988346,2023-04-25,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,17.0,6,6,1.0,0.182759,0.644737,0.034483,0.131579,0.592105,1.315789
11988347,2023-04-26,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,22.0,6,0,0.0,0.157143,0.757576,0.035714,0.151515,0.757576,1.515152
11988348,2023-04-27,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,22.0,6,6,0.0,0.102941,1.166667,0.000000,0.208333,1.041667,2.083333
11988349,2023-04-28,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,28.0,6,18,2.0,0.063462,1.120000,0.000000,0.200000,1.000000,2.000000
11988350,2023-04-29,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,32.0,0,18,5.0,0.062000,0.913793,0.000000,0.172414,0.862069,1.724138
11988351,2023-04-30,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,27.0,0,24,14.0,0.058824,0.785714,0.000000,0.178571,0.803571,1.607143


# Escrever

- 0's e 1's

In [45]:
# Novo dataframe que apenas inclui dias em que existe a 1ª rotura e o dia anterior a essa rotura

# Tirar 1's depois do primeiro
dfFinalLimitado = dfFinal[~((dfFinal["ROTURA"] == 1) & (dfFinal["ROTURA"].shift(1) == 1))].copy()

# Apenas incluir primeira rotura
dfFinalLimitadoRoturas = dfFinal[((dfFinal["ROTURA"] == 1) & (dfFinal["ROTURA"].shift(1) == 0))].copy() #| ((dfFinal["ROTURA"] == 0) & (dfFinal["ROTURA"].shift(-1) == 1))]

- Dias certos

dfFinal = dfFinal.loc[(dfFinal['DATA'] >= '2023-01-01') ].copy()
df_DiasCertos = dfFinalLimitado.loc[(dfFinalLimitado['DATA'] >= '2023-01-01') ].copy()
df_RoturasDiasCertos = dfFinalLimitadoRoturas.loc[(dfFinalLimitadoRoturas['DATA'] >= '2023-01-01') ].copy()

- Passar para csv

In [46]:
%%time

escrever_csv(dfFinal, "Stocks_Delta_2022_10prod_Métricas")
escrever_csv(dfFinalLimitado, "Stocks_Delta_2022_10prod_Métricas_Limpo")
escrever_csv(dfFinalLimitadoRoturas, "Stocks_Delta_2022_10prod_Métricas_SóRoturas")

CPU times: total: 4min 24s
Wall time: 5min 20s
